In [403]:
import pandas as pd
import numpy as np
import datetime
import os

from DataTool import DataLoader
data_loader = DataLoader()
analysis_folder_name = 'german_play'
baseline_csv_name = 'german.csv'

w_mean   = 1
w_std    = 0.0

w_recall = 0.0
w_auc    = 1.0
w_gmean  = 0.0

best_result_pick_num = 5
pd.set_option('display.float_format',lambda x : '%.5f' % x)

In [404]:
baseline = data_loader.load_csv_to_pandas(f'./result/baseline/{baseline_csv_name}')
results  = data_loader.load_csv_to_pandas(f'./result/{analysis_folder_name}/result.csv')

In [405]:
baseline

,resampler,recall_mean,recall_std,auc_mean,auc_std,gmean_mean,gmean_std
0,adasyn,0.59512,0.13270,0.66614,0.03937,0.64547,0.06375
1,basic_smote,0.59146,0.12729,0.65720,0.03470,0.63758,0.05702
2,bordered_smote,0.58049,0.14096,0.65492,0.02591,0.63207,0.05256
3,instance_hardness_threshold,0.83780,0.03452,0.67739,0.02055,0.65797,0.01869
4,MWMOTE,0.58293,0.14834,0.65821,0.03275,0.63509,0.05748
5,near_miss,0.65122,0.05520,0.62263,0.01923,0.62048,0.01924
6,no_resampling,0.47805,0.17254,0.63306,0.02548,0.58872,0.05129
7,one_sided_selection,0.52683,0.15110,0.65241,0.02914,0.61958,0.05062
8,random_over_sampling,0.59756,0.14880,0.65062,0.03594,0.62967,0.06412
9,random_under_sampling,0.72073,0.08099,0.67894,0.03495,0.66720,0.06858


In [406]:
print(results.shape)
# drop columns
drop_columns = ['resample_time', 'train_time', 'support', 'shuffle', 'original_maj_min', 'train_args']
results = results.drop(drop_columns, axis=1)
# results.head(5)

(180, 17)


In [407]:
#! results先根据id排序
sorted_results = results.sort_values(by='id')
# sorted_results.head(15)

In [408]:
# group by args
grouped_results = sorted_results.groupby(['resample_args']) \
                                .agg(recall_mean=('recall', 'mean'),
                                     recall_std=('recall', 'std'),
                                     auc_mean=('auc', 'mean'),
                                     auc_std=('auc', 'std'),
                                     gmean_mean=('gmean', 'mean'),
                                     gmean_std=('gmean', 'std'))
  
# grouped_results


In [409]:
#! add rank info
# grouped_results['recall_mean_rank']  =  grouped_results['recall_mean'].rank(method='dense', ascending=False)
# grouped_results['recall_std_rank']   =  grouped_results['recall_std'].rank(method='dense', ascending=False)
# grouped_results['auc_mean_rank']     =  grouped_results['auc_mean'].rank(method='dense', ascending=False)
# grouped_results['auc_std_rank']      =  grouped_results['auc_std'].rank(method='dense', ascending=False)
# grouped_results['gmean_mean_rank']   =  grouped_results['gmean_mean'].rank(method='dense', ascending=False)
# grouped_results['gmean_std_rank']    =  grouped_results['gmean_std'].rank(method='dense', ascending=False)
grouped_results['recall_mean_norm']  =  (grouped_results['recall_mean'] - grouped_results['recall_mean'].min()) / (grouped_results['recall_mean'].max() - grouped_results['recall_mean'].min())
grouped_results['recall_std_norm']   =  (grouped_results['recall_std']  - grouped_results['recall_std'].max())  / -(grouped_results['recall_std'].max()  - grouped_results['recall_std'].min())
grouped_results['auc_mean_norm']     =  (grouped_results['auc_mean']    - grouped_results['auc_mean'].min())    / (grouped_results['auc_mean'].max()    - grouped_results['auc_mean'].min())
grouped_results['auc_std_norm']      =  (grouped_results['auc_std']     - grouped_results['auc_std'].max())     / -(grouped_results['auc_std'].max()     - grouped_results['auc_std'].min())
grouped_results['gmean_mean_norm']   =  (grouped_results['gmean_mean']  - grouped_results['gmean_mean'].min())  / (grouped_results['gmean_mean'].max()  - grouped_results['gmean_mean'].min())
grouped_results['gmean_std_norm']    =  (grouped_results['gmean_std']   - grouped_results['gmean_std'].max())   / -(grouped_results['gmean_std'].max()   - grouped_results['gmean_std'].min())
grouped_results['score']             =  w_recall * ( w_mean * grouped_results['recall_mean_norm'] + w_std * grouped_results['recall_std_norm']) + \
                                        w_auc    * ( w_mean * grouped_results['auc_mean_norm']    + w_std * grouped_results['auc_std_norm'   ]) + \
                                        w_gmean  * ( w_mean * grouped_results['gmean_mean_norm']  + w_std * grouped_results['gmean_std_norm' ])
grouped_results = grouped_results.sort_values(by='score', ascending=False)
grouped_results.head(best_result_pick_num)

,recall_mean,recall_std,auc_mean,auc_std,gmean_mean,gmean_std,recall_mean_norm,recall_std_norm,auc_mean_norm,auc_std_norm,gmean_mean_norm,gmean_std_norm,score
resample_args,,,,,,,,,,,,,
"{'a_ros': 1.5, 'a_rus': 1.5, 'i_ros': 0.1}",0.69634,0.09152,0.68349,0.04267,0.67436,0.06340,1.00000,0.59619,1.00000,0.25529,1.00000,0.37173,1.00000
"{'a_ros': 0.5, 'a_rus': 1, 'i_ros': 0.2}",0.67805,0.08646,0.67251,0.03909,0.66054,0.07209,0.58333,0.72871,0.65973,0.39359,0.59334,0.12368,0.65973
"{'a_ros': 1.5, 'a_rus': 0.5, 'i_ros': 0.2}",0.67805,0.08665,0.67228,0.03276,0.66438,0.05065,0.58333,0.72371,0.65263,0.63814,0.70623,0.73598,0.65263
"{'a_ros': 1, 'a_rus': 1.5, 'i_ros': 0.1}",0.65244,0.07610,0.67209,0.03892,0.67053,0.04140,0.00000,1.00000,0.64673,0.40028,0.88726,1.00000,0.64673
"{'a_ros': 0.5, 'a_rus': 0.5, 'i_ros': 0.2}",0.66707,0.10491,0.66978,0.04383,0.65565,0.07642,0.33333,0.24543,0.57497,0.21052,0.44931,-0.00000,0.57497


In [410]:
# slice_col  = ['recall_mean', 'recall_std', 'auc_mean', 'auc_std', 'gmean_mean', 'gmean_std']
best_result = grouped_results.iloc[:best_result_pick_num, 0:6]
best_result['resampler'] = grouped_results.index[:best_result_pick_num]
best_result

,recall_mean,recall_std,auc_mean,auc_std,gmean_mean,gmean_std,resampler
resample_args,,,,,,,
"{'a_ros': 1.5, 'a_rus': 1.5, 'i_ros': 0.1}",0.69634,0.09152,0.68349,0.04267,0.67436,0.06340,"{'a_ros': 1.5, 'a_rus': 1.5, 'i_ros': 0.1}"
"{'a_ros': 0.5, 'a_rus': 1, 'i_ros': 0.2}",0.67805,0.08646,0.67251,0.03909,0.66054,0.07209,"{'a_ros': 0.5, 'a_rus': 1, 'i_ros': 0.2}"
"{'a_ros': 1.5, 'a_rus': 0.5, 'i_ros': 0.2}",0.67805,0.08665,0.67228,0.03276,0.66438,0.05065,"{'a_ros': 1.5, 'a_rus': 0.5, 'i_ros': 0.2}"
"{'a_ros': 1, 'a_rus': 1.5, 'i_ros': 0.1}",0.65244,0.07610,0.67209,0.03892,0.67053,0.04140,"{'a_ros': 1, 'a_rus': 1.5, 'i_ros': 0.1}"
"{'a_ros': 0.5, 'a_rus': 0.5, 'i_ros': 0.2}",0.66707,0.10491,0.66978,0.04383,0.65565,0.07642,"{'a_ros': 0.5, 'a_rus': 0.5, 'i_ros': 0.2}"


In [411]:
baseline = pd.concat([baseline, best_result], axis=0)
# baseline

In [412]:
baseline['recall_mean_norm']  =  (baseline['recall_mean'] - baseline['recall_mean'].min()) / (baseline['recall_mean'].max() - baseline['recall_mean'].min())
baseline['recall_std_norm']   =  (baseline['recall_std']  - baseline['recall_std'].max())  / -(baseline['recall_std'].max()  - baseline['recall_std'].min())
baseline['auc_mean_norm']     =  (baseline['auc_mean']    - baseline['auc_mean'].min())    / (baseline['auc_mean'].max()    - baseline['auc_mean'].min())
baseline['auc_std_norm']      =  (baseline['auc_std']     - baseline['auc_std'].max())     / -(baseline['auc_std'].max()     - baseline['auc_std'].min())
baseline['gmean_mean_norm']   =  (baseline['gmean_mean']  - baseline['gmean_mean'].min())  / (baseline['gmean_mean'].max()  - baseline['gmean_mean'].min())
baseline['gmean_std_norm']    =  (baseline['gmean_std']   - baseline['gmean_std'].max())   / -(baseline['gmean_std'].max()   - baseline['gmean_std'].min())
baseline['score']             =  w_recall * ( w_mean * baseline['recall_mean_norm']  + w_std * baseline['recall_std_norm']) + \
                                 w_auc    * ( w_mean * baseline['auc_mean_norm']     + w_std * baseline['auc_std_norm'   ]) + \
                                 w_gmean  * ( w_mean * baseline['gmean_mean_norm']   + w_std * baseline['gmean_std_norm' ])
baseline = baseline.sort_values(by='score', ascending=False)
baseline

,resampler,recall_mean,recall_std,auc_mean,auc_std,gmean_mean,gmean_std,recall_mean_norm,recall_std_norm,auc_mean_norm,auc_std_norm,gmean_mean_norm,gmean_std_norm,score
"{'a_ros': 1.5, 'a_rus': 1.5, 'i_ros': 0.1}","{'a_ros': 1.5, 'a_rus': 1.5, 'i_ros': 0.1}",0.69634,0.09152,0.68349,0.04267,0.67436,0.06340,0.60678,0.58699,1.00000,0.04710,1.00000,0.22550,1.00000
9,random_under_sampling,0.72073,0.08099,0.67894,0.03495,0.66720,0.06858,0.67458,0.66333,0.92528,0.36116,0.91636,0.13586,0.92528
3,instance_hardness_threshold,0.83780,0.03452,0.67739,0.02055,0.65797,0.01869,1.00000,1.00000,0.89972,0.94643,0.80857,1.00000,0.89972
"{'a_ros': 0.5, 'a_rus': 1, 'i_ros': 0.2}","{'a_ros': 0.5, 'a_rus': 1, 'i_ros': 0.2}",0.67805,0.08646,0.67251,0.03909,0.66054,0.07209,0.55593,0.62365,0.81958,0.19257,0.83865,0.07503,0.81958
"{'a_ros': 1.5, 'a_rus': 0.5, 'i_ros': 0.2}","{'a_ros': 1.5, 'a_rus': 0.5, 'i_ros': 0.2}",0.67805,0.08665,0.67228,0.03276,0.66438,0.05065,0.55593,0.62227,0.81581,0.44981,0.88344,0.44647,0.81581
"{'a_ros': 1, 'a_rus': 1.5, 'i_ros': 0.1}","{'a_ros': 1, 'a_rus': 1.5, 'i_ros': 0.1}",0.65244,0.07610,0.67209,0.03892,0.67053,0.04140,0.48475,0.69870,0.81268,0.19960,0.95527,0.60663,0.81268
"{'a_ros': 0.5, 'a_rus': 0.5, 'i_ros': 0.2}","{'a_ros': 0.5, 'a_rus': 0.5, 'i_ros': 0.2}",0.66707,0.10491,0.66978,0.04383,0.65565,0.07642,0.52542,0.48997,0.77463,-0.00000,0.78150,-0.00000,0.77463
0,adasyn,0.59512,0.13270,0.66614,0.03937,0.64547,0.06375,0.32542,0.28868,0.71489,0.18123,0.66261,0.21948,0.71489
4,MWMOTE,0.58293,0.14834,0.65821,0.03275,0.63509,0.05748,0.29153,0.17536,0.58456,0.45061,0.54142,0.32805,0.58456
1,basic_smote,0.59146,0.12729,0.65720,0.03470,0.63758,0.05702,0.31525,0.32786,0.56801,0.37097,0.57055,0.33617,0.56801
